In [ ]:
from tools import *

# Synopsis
Dans cet atelier nous mettons en pratique les concepts vus ce mardi. Pour cela,
nous analysons une base de données publique fournissant une estimation des
émissions de carbonne chaque année (sur base de la consommation de carburants
fossiles et de la production de ciment) entre 1751 et 2014 [1].

Les données sont fournies sous forme de deux listes `X` et `Y`. La liste `X`
contient les années de mesures alors que la liste `Y` contient les émissions de
carbonne, exprimées en millions de tonnes. L'entrée en position `i` de la liste
`X` correspond à l'année des émissions de carbonne à la position `i` de la liste
`Y`. Par exemple si `X[i] = 2000` et `Y[i] = 6733`, cela signifie que 6733
millions de tonnes de carbonne ont été émises en l'an 2000.

La liste `X` est ordonnée en ordre strictement croissant telle que pour tout
`i >= 0`,
* si `j > i` et `j < len(X)` alors `X[j] > X[i]`
* si `j = i` et `j < len(X)` alors `X[j] = X[i]`.

La combinaison de `X` et `Y` forment ce qu'on appelle courrement une _série
temporelle_ [2].

Nous commençerons par afficher les données sous forme de graphique et à y
apporter quelques interpretations. Ensuite, nous modeliserons la tendance de
cette série temporelle par une fonction de croissance exponentielle et
utiliserons ce modèle pour fournir une analyse circonstanciée de l'évolution des
émissions de carbonne ces derniers siècles. Finalement, nous proposerons une
ébauche de validation de notre modèle et en nuancerons les conclusions.

# Chargement des données
Utilisons la fonction `loadData` pour charger la base de données en mémoire et
accéder à ces information via les listes `X` et `Y`. N'hésitez pas à regarder
les fichiers `tools.py` et `global.1751_2014.csv` pour comprendre comment il est
possible de stocker des informations et de les extraire de manière systématique.

Pour rendre notre code plus générique utilisons une constante contenant le nom
du ficher contenant les données brutes.

In [ ]:
FICHIER_DONNEES_BRUTES = 'global.1751_2014.csv' # ficher de données à charger
X, Y = loadData(FICHIER_DONNEES_BRUTES)         # charger les données en mémoire

Les données qui composent notre série temporelle étant chargées en mémoire nous
pouvons les visualiser grâce à un graphe. Même si les données dont nous
disposons sont fondamentalement discrètes, il est d'usage de représenter les 
séries temporelles avec une ligne continue passant par tous les couples
_(temps, valeur)_ issus de la base de donnée.

Nous allons utiliser la librairie `matplotlib` de Python [3]. Cette librairie
permet de représenter de manière graphique des jeux de données arbitraires. Plus
précisément nous allons utiliser la fonction `pyplot` de cette librairie [4].
Pour nous simplifier la tâche, nous l'avons déjà importée et pouvous l'appeler
n'importe où dans le code avec nom `plt`.

Voici 4 fonctions de `pyplot` qui vont nous servir pour représenter nos données
de manière graphique:
* `plt.pot(x, y, label)` qui trace la courbe où `x` est une liste de point en
abscisse et `y` est la liste de points en ordonnées correspondant aux abscisse.
`label` est le nom de la courbe.

* `plt.xlabel(texte)` permet d'indiquer le nom de l'axe des abscisse.
* `plt.ylabel(texte)` permet d'indiquer le nom de l'axe des ordonnées.
* `plt.legend()` demande d'afficher la légende.

L'exemple ci-dessous, montre comment utiliser la fonction `plt` pour tracer une
courbe.

In [ ]:
abscisse = [1,2,3,4]
ordonnees = [10,5, 30, 20]
plt.plot(abscisse, ordonnees, label='Points arbitraires')
plt.xlabel('Axe des x')
plt.ylabel('Axe des y')
plt.legend()

Utilisez la fonction `plt` pour représenter de manière graphique nos données `X`
et `Y`. L'axe des abscisse devrait être nommé `Année` alors que l'axe des
ordonnées devrait être nommé `Emissions fossiles en MtC`. `MtC` signifiant
million de tonnes d'émissions de carbonne ici. La courbe devrait avoir pour
légende `Données empiriques`.


Pourquoi lorsque nous représentons des données il est indispensable de toujours
nommer données et leurs axes ainsi que de préciser dans quelles unités les
données sont exprimées?

In [ ]:
plt.plot(X, Y, label='Données empiriques')  # tracer la courbe
plt.xlabel('Année')                         # nommer l'axe des x
plt.ylabel('Emissions fossiles MtC')        # nommer l'axe des y
plt.legend()                                # afficher la légende

Commentez la courbe obtenue en plaçant dans le contexte historique.

Lorsque l'on travail avec des bases de données, il est souvent intéressant de se
concentrer sur un sous-ensemble des données de la base.

Pour la suite de notre analyse, nous allons limiter notre étude à certaines
périodes de temps. Pour simplifier le raisonnement sur les données nous allons
appliquer un principe fondamental de la programation: _la décomposition en
sous-problèmes_.

Créons donc la fonction `donneesPeriode(X, Y, debut, fin)`. `X` et `Y` sont les
valeurs en x et y des données brutes. `debut` est la date de début souhaitée
de la période à extraire des données brutes et `fin` est la date de fin de cette
période. La fonction retourne `X_` et `Y_` deux sous-listes extraites de `X` et
`Y`. Notre hypothèse de travail ici est que les données brutes contiennent
toutes les années, classées de manière croissante.

Si `X = [2001, 2002, 2003, 2004]` et `Y = [1, 5, 3, 6]` alors `X_` et `Y_`
obtenus par l'appel `X_, Y_ = donneesPeriode(X, Y, 2002, 2003)`  devraient
valoir `[2002, 2003]` et `[5, 3]`, respectivement.

Python offre une syntaxe simple pour extraire une sous-liste d'une liste
simplement en donnant l'index du début de la sous-liste et son indexe de fin,
séparés par le symbole `:`. Ainsi l'expression `liste[i:j]` retourne la
sous-liste constituée des éléments de la liste `liste`, commençant à l'index `i`
inclu et terminant à l'indice `j` exclu. Python, comme la majorité des languages
de programmation, indexe ses listes en partant de 0. Dans l'exemple ci-dessus,
`X[0]` donnerait donc la valeur 2001 alors que `X[2]` donnerait `2003`.

Pour extraire la sous-liste commençant à l'année 2002 et terminant à l'année 2003
il nous suffit donc de faire `X_ = X[1:3]` et `Y_ = Y[1:3]` car l'année 2002 se
trouve à l'index 1 et l'année 2003 se trouve à l'indexe 2 de `X`.

La difficulté réside donc ici à trouver les indexes de début et de fin de la
sous-liste qui nous intéresse.

In [ ]:
def donneesPeriode(X, Y, debut, fin):
    """
    X: années du jeu de données
    Y: emission du jeu de données
    debut: date de début des données à extraire
    fin: date de fin des données à extraire
    """
    nb_mesures = fin - debut + 1                # nombre de mesures à extraire

    index_initial = debut - X[0]                # index de la première mesure
    index_final = index_initial + nb_mesures    # index de la dernière mesure

    X_ = X[index_initial:index_final]           # garder les x sur l'intervalle 
    Y_ = Y[index_initial:index_final]           # garder les y sur l'intervalle

    return X_, Y_                               # retourner les x et les y

# Modélisation

## Modèle de croissance exponentielle
Le modèle exponentiel prend un taux de croissance et une valeur initiale.

La valeur au bout de _n_ itérations se calcule selon la formule générale _y_n = C^n * y_0_

Sur base d'un modèle de croissance exponentielle, trouver la valeur _y_ au bout de
 _n_ années depuis le début des mesures.

In [ ]:
def modeleExponentiel(croissance, y0, n):
    """
    croissance: taux de croissance annuel exprimé en %
    y0: valeur initiale
    n: itération à laquelle prévoir la valeur
    """
    C = 1.0 + (croissance / 100.0)  # facteur de croissance

    yn = (C ** n) * y0              # valeur au bout de n années

    return yn                       # retourner la valeur après n années

## Estimation du taux de croissance annuelle
Estimation du taux de croissance annuel en supposant une evolution exponentielle
des émissions.

Utiliser un ajustement de courbe pour déterminer les paramètres du modèle de
croissance exponentielle.

In [ ]:
def parametresModeleExponentiel(X, Y):
    """
    X: _x_ connus
    Y: _y_ connus
    """
    x0 = X[0]                       # première année de mesures
    y0 = Y[0]                       # valeur des émissions lors de cette année

    xn = X[len(X) - 1]              # dernière année de mesures
    yn = Y[len(X) - 1]              # valeur des émissions lors de cette année

    nb_annees = xn - x0             # nombre d'années de mesures

    C = (yn / y0)**(1.0/nb_annees)  # facteur de croissance
    
    croissance = (C - 1) * 100.0    # taux de croissance annuel en %

    return y0, croissance           # retourner la valeur initiale et le taux

Calculer le modèle sur un interval défini

In [ ]:
def parametresModeleExponentielPeriode(X, Y, debut, fin):
    """
    X: données brutes
    Y: données brutes
    debut: date de debut de l'intervalle d'analyse
    fin: date de fin de l'intervalle d'analyse
    """
    X_, Y_ = donneesPeriode(X, Y, debut, fin)   # données sur l'intervalle

    # obtenir les parametres de l'exponentielle
    y0, croissance = parametresModeleExponentiel (X_, Y_) 

    # calculer toutes les émissions dans l'intervalle de temps grace au modèle
    # exponentiel
    Y_modele = [modeleExponentiel(croissance, y0, i) for i in range(len(X_))]
    
    return X_, Y_, Y_modele, croissance # retourner les données calculées

## Taux de croissance sur l'ensemble des données

### Calcul du taux de croissance sur différents intervalles de temps
1. l'entiereté des données
2. la période [1850, 1913]
3. la période [1945, 1973]

In [ ]:
plt.plot(X, Y, label='Données empiriques')  # afficher les données sous forme de
                                            # graphe
plt.xlabel('Année')                         # nommer l'axe des x
plt.ylabel('Emission fossiles GtC')         # nommer l'axe des y

periodes = [
    (X[0], X[len(X)-1]),                    # l'entiereté des données
    (1850, 1913),                           # [1850, 1913]
    (1945, 1973)                            # [1945, 1973]
    ]                                       # lister les intervalles à étudier

croissances = dict()                        # taux de croissance calculés pour
                                            # chaque période
                                            #   clé = période
                                            #   valeur = taux de croissance

# représenter graphiquement les résultats pour tous les intervalles de temps et
# mémoriser le taux de croissance annuel
for debut, fin in periodes:
    # calculer les parametres du modèle sur l'intervalle de temps
    X_, Y_, Y_modele, croissance = parametresModeleExponentielPeriode(X, Y, debut, fin)

    # mémoriser le taux de croissance
    croissances[(debut, fin)] = croissance

    # ajouter la courbe à la figure
    plt.plot(X_, Y_modele,
             label='Modèle exponentiel [{debut}, {fin}] ({croissance:.2f}%)'
                    .format(debut=debut, fin=fin, croissance=croissance))

plt.legend()

# Quel est le temps de doublement?

Astuce:
* Produit: log(M * N) = log(M) + log(N)
* Quotient: log (M/N) = log(M) - log (N)
* Puissance: log(M^p) = p * log(M)
pour M > 0 et N > 0

In [ ]:
def temps_doublement(croissance):
    C = 1.0 + (croissance / 100.0)      # facteur de croissance

    temps = log(2) / log (C)            # calcul du temps de doublement

    return temps                        # retourner le temps de doublement

In [ ]:
doublements = dict()    # temps de doublement calculés pour chaque période
                        #   clé = période
                        #   valeur = temps de doublement

for (periode, value) in croissances.items():
    # calculer le temps de doublement (en années entières)
    temps = int(temps_doublement(croissances[periode]))

    # mémorier le temps de doublement
    doublements[periode] = temps

    # afficher le temps de doublement
    print ("Temps de doublement pour la période [{debut}, {fin}]: {temps} années"
          .format(debut=periode[0], fin=periode[1], temps = temps))

## Validation des valeurs obtenues

### Analyse des émissions à un momment donné

Comparaison avec les données brutes

In [ ]:
periode = periodes[0]               # période depuis le début des données
(debut, fin) = periode              # années de début et fin de la période

fin_periode_double = fin            # fin de la dernière période de doublement
debut_periode_double = fin - doublements[periode] # début de la dernière période
                                                  # de doublement

# obtention des données sur la dernière période de doublement
X_periode_double, Y_period_double = donneesPeriode(X, Y, debut_periode_double, fin_periode_double)

y_debut_periode = Y_period_double[0]                        # emissions au début
                                                            # de la période
y_fin_periode = Y_period_double[len(Y_period_double) - 1]   # émissions à la fin
                                                            # de la période

# afficher le rapport entre les émissions de la période 
print (y_fin_periode / y_debut_periode)

Comparaison avec le modèle

In [ ]:
croissance = croissances[periode]   # récupérer le taux de croissance sur la
                                    # période

n_debut_periode_double = debut_periode_double - debut # nombre d'années écoulées
                                                      # entre le début des
                                                      # mesures et le début de
                                                      # la dernière période de
                                                      # doublement
n_fin_periode_double = fin_periode_double - debut     # nombre d'années écoulées
                                                      # entre le début des
                                                      # mesures et la fin de la
                                                      # dernière période de
                                                      # doublement

y_debut_periode_modele = modeleExponentiel(croissance, Y[0], n_debut_periode_double) # calculer la valeur de l'expontentielle au début de la dernière période de doublement
y_fin_periode_modele = modeleExponentiel(croissance, Y[0], n_fin_periode_double)     # calculer la valeur de l'expontentielle à la fin de la dernière période de doublement


# afficher le rapport entre les émission de la période
print (y_fin_periode_modele / y_debut_periode_modele)

### Analyse des cumuls d'émissions

Comparaison avec les données brutes

In [ ]:
emissions_periode_double = sum(Y_period_double)     # calculer la somme des
                                                    # émissions sur la dernière
                                                    # période de doublement

# obtention des données sur la période avant la période de doublement
X_avant, Y_avant = donneesPeriode(X, Y, debut, debut_periode_double - 1)

emissions_avant = sum(Y_avant)      # calculer la somme des émissions sur la
                                    # période avant la dernière période de
                                    # doublement

# afficher le rapport entre la somme des émissions de la dernière période
# d'émition et de la période la précédent
print (emissions_periode_double/emissions_avant)


Comparaison avec le modèle

In [ ]:
# calculer la somme des émissions anticipées par le modèle de croissance
# exponentiel sur la dernière période de doublement

emissions_periode_double = sum ([modeleExponentiel(croissance, Y_avant[0], n) for n in range(debut_periode_double - debut, fin_periode_double - debut)])

# calculer la somme des émissions anticipées par le modèle de croissance
# exponentiel sur la période avant la dernière période de doublement
emissions_avant = sum ([modeleExponentiel(croissance, Y_avant[0], n) for n in range(debut - debut, debut_periode_double - debut)])

# afficher le rapport entre la somme des émissions de la dernière période
# d'émition et de la période la précédent
print (emissions_periode_double / emissions_avant)

### Comment pouvez-vous expliquez ces différences ?


# Aller plus loin
* Trouvez d'autres sources de données publiques et comparez les résultats
obtenus. Si les données que vous trouvez sont plus récentes, emettez des
prévisions d'émissions sur bases de nos données et comparez les aux observations
plus récentes.

# References
[1] Boden, T.A., G. Marland, and R.J. Andres. 2017. Global, Regional, and National Fossil-Fuel CO2 Emissions. Carbon Dioxide Information Analysis Center, Oak Ridge National Laboratory, U.S. Department of Energy, Oak Ridge, Tenn., U.S.A. doi 10.3334/CDIAC/00001_V2017.

[2] Louis, F. 2021. Daniel , peux-tu nous parler des séries temporelles ?, DataScientest, https://datascientest.com/series-temporelles.

[3] Matplotlib: Visualization with Python. 2022. https://matplotlib.org/.

[4] Pyplot function overview. 2022. https://matplotlib.org/stable/api/pyplot_summary.html.